In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

In [9]:
# Lee el diccionario de regiones
diccionario_regiones = (
    pd.read_excel("../data/external/Esquema_Registro-2023.xlsx", sheet_name="Anexo 2", header=6)
    .iloc[:, 5:9]
    .dropna(subset="Código Comuna")
)

diccionario_regiones["Código Comuna"] = (
    diccionario_regiones["Código Comuna"].astype(int).astype(str)
)

In [10]:
diccionario_regiones

,Código Comuna,Nombre Comuna,Código Región,Nombre Región
0,15101,ARICA,15.0,de Arica y Parinacota
1,15102,CAMARONES,15.0,de Arica y Parinacota
2,15201,PUTRE,15.0,de Arica y Parinacota
3,15202,GENERAL LAGOS,15.0,de Arica y Parinacota
4,1101,IQUIQUE,1.0,de Tarapacá
...,...,...,...,...
343,12303,TIMAUKEL,12.0,de Magallanes y de la Antártica Chilena
344,12401,NATALES,12.0,de Magallanes y de la Antártica Chilena
345,12402,TORRES DEL PAINE,12.0,de Magallanes y de la Antártica Chilena
346,88888,Extranjero,88.0,Extranjero


In [13]:
df = pd.read_excel(
    "../data/raw/1_poblacion_ine/censo_2024/DATOS COMUNA PARA POBLACION POR EDAD 25.06.2025.xlsx"
)

In [14]:
df["CÓDIGO COMUNA"] = df["CÓDIGO COMUNA"].astype(str)
df = df.merge(diccionario_regiones, how="left", left_on="CÓDIGO COMUNA", right_on="Código Comuna")

In [24]:
columnas_hombres = df.columns[df.columns.str.contains("HOMBRES")]
columnas_mujeres = df.columns[df.columns.str.contains("MUJERES")]
columnas_valores = list(columnas_hombres) + list(columnas_mujeres)

In [30]:
formato_largo_poblacion = pd.melt(
    df,
    id_vars=["Código Región", "Nombre Región", "Código Comuna", "Nombre Comuna", "Edad"],
    value_vars=columnas_valores,
)

,NOMBRE COMUNA,CÓDIGO COMUNA,Edad,edad_inicio,edad_fin,tamaño_grupo,Población censada,Hombres,Mujeres,2024 por tramo,2025 por tramo,2026 por tramo,2027 por tramo,2028 por tramo,2029 por tramo,2030 por tramo,2031 por tramo,2032 por tramo,2033 por tramo,2034 por tramo,2035 por tramo,2036 por tramo,2037 por tramo,2038 por tramo,2039 por tramo,2040 por tramo,2041 por tramo,2042 por tramo,2043 por tramo,2044 por tramo,lista_Edad,poblacion censada / tamaño grupo,poblacion hombres/ tamaño grupo,poblacion mujeres/tamaño grupo,Proporción Hombres,Proporción Mujeres,2024 por edad,2025 por edad,2026 por edad,2027 por edad,2028 por edad,2029 por edad,2030 por edad,2031 por edad,2032 por edad,2033 por edad,2034 por edad,2035 por edad,2036 por edad,2037 por edad,2038 por edad,2039 por edad,2040 por edad,2041 por edad,2042 por edad,2043 por edad,2044 por edad,HOMBRES 2024,HOMBRES 2025,HOMBRES 2026,HOMBRES 2027,HOMBRES 2028,HOMBRES 2029,HOMBRES 2030,HOMBRES 2031,HOMBRES 2032,HOMBRES 2033,HOMBRES 2034,HOMBRES 2035,HOMBRES 2036,HOMBRES 2037,HOMBRES 2038,HOMBRES 2039,HOMBRES 2040,HOMBRES 2041,HOMBRES 2042,HOMBRES 2043,HOMBRES 2044,MUJERES 2024,MUJERES 2025,MUJERES 2026,MUJERES 2027,MUJERES 2028,MUJERES 2029,MUJERES 2030,MUJERES 2031,MUJERES 2032,MUJERES 2033,MUJERES 2034,MUJERES 2035,MUJERES 2036,MUJERES 2037,MUJERES 2038,MUJERES 2039,MUJERES 2040,MUJERES 2041,MUJERES 2042,MUJERES 2043,MUJERES 2044,Código Comuna,Nombre Comuna,Código Región,Nombre Región
0,PAÍS,País,0,0,4,5,870693,443028,427665,870693,878129.183957,885759.00264,893587.554656,901620.169309,909862.413401,918320.098581,926999.28926,935906.311094,945047.760035,954430.511976,964061.732993,973948.890195,984099.763213,994522.456334,1.005225e+06,1.016217e+06,1.027508e+06,1.039106e+06,1.051021e+06,1.063265e+06,0,174138.600000,88605.6,85533.000000,0.508822,0.491178,174138.600000,175625.836791,177151.800528,178717.510931,180324.033862,181972.48268,183664.019716,185399.857852,187181.262219,189009.552007,190886.102395,192812.346599,194789.778039,196819.952643,198904.491267,201045.082262,203243.484169,205501.528565,207821.123056,210204.254421,212652.991923,88605.6,89362.339219,90138.783572,90935.451913,91752.886579,92591.65407,93452.345806,94335.578929,95241.997166,96172.27175,97127.102402,98107.218375,99113.379555,100146.377632,101207.037333,102296.21773,103414.813607,104563.756912,105744.018277,106956.608618,108202.580824,85533.000000,86263.497572,87013.016956,87782.059018,88571.147283,89380.828611,90211.673910,91064.278923,91939.265053,92837.280257,93758.999993,94705.128223,95676.398484,96673.575011,97697.453933,98748.864531,99828.670561,100937.771653,102077.104779,103247.645803,104450.411099,NaN,NaN,NaN,NaN
1,PAÍS,País,1,0,4,5,870693,443028,427665,870693,878129.183957,885759.00264,893587.554656,901620.169309,909862.413401,918320.098581,926999.28926,935906.311094,945047.760035,954430.511976,964061.732993,973948.890195,984099.763213,994522.456334,1.005225e+06,1.016217e+06,1.027508e+06,1.039106e+06,1.051021e+06,1.063265e+06,1,174138.600000,88605.6,85533.000000,0.508822,0.491178,174138.600000,175625.836791,177151.800528,178717.510931,180324.033862,181972.48268,183664.019716,185399.857852,187181.262219,189009.552007,190886.102395,192812.346599,194789.778039,196819.952643,198904.491267,201045.082262,203243.484169,205501.528565,207821.123056,210204.254421,212652.991923,88605.6,89362.339219,90138.783572,90935.451913,91752.886579,92591.65407,93452.345806,94335.578929,95241.997166,96172.27175,97127.102402,98107.218375,99113.379555,100146.377632,101207.037333,102296.21773,103414.813607,104563.756912,105744.018277,106956.608618,108202.580824,85533.000000,86263.497572,87013.016956,87782.059018,88571.147283,89380.828611,90211.673910,91064.278923,91939.265053,92837.280257,93758.999993,94705.128223,95676.398484,96673.575011,97697.453933,98748.864531,99828.670561,100937.771653,102077.104779,103247.645803,104450.411099,NaN,NaN,NaN,NaN
2,PAÍS,País,2,0,4,5,870693,443028,427665,870693,878129.183957,885759.00264